In [ ]:
import replicate
import requests
import shutil
import uuid
import random
from IPython.core.display import Image, display

OUTPUT_DIR = "/tmp/replicate/"
PATH_TO_PIC_OF_YOUR_CAT = "/home/tren/Pictures/zumolabs/cat_package_edit.png"


In [ ]:
# Predict the species of Buboo
model = replicate.models.get("replicate/resnet")
model.predict(image=open(PATH_TO_PIC_OF_YOUR_CAT, "rb"))

In [ ]:
# Generative model
gen_model = replicate.models.get("prompthero/openjourney")
gen_version = gen_model.versions.get(
    "9936c2001faa2194a261c01381f90e65261879985476014a0a37a334593a05eb")

# Upscaling model
upscale_model = replicate.models.get("jingyunliang/swinir")
upscale_version = upscale_model.versions.get(
    "660d922d33153019e8c263a3bba265de882e7f4f70396546b6c9c8f9d47a021a")

actual_prompt = "engineer inventor tinkering with robot in workshop, "

styles = [
    "mdjrny-v4 style, ",
    "by studio ghibli, ",
    "makoto shinkai, ",
    "by artgerm, ",
    "by wlop, ",
    "by greg rutkowski, ",
    "by jason chan, ",
    "volumetric lighting, ",
    "octane render, ",
    "smooth, ",
    "sharp focus, ",
    "intricate, ",
    "Chromatic Aberration, ",
    "insanely detailed and intricate, "
    "hypermaximalist, ",
    "elegant, ",
    "ornate, ",
    "hyper realistic, ",
    "super detailed, ",
    "Unreal Engine, ",
    "4k resolution, ",
    "trending on artstation, ",
]

num_inference_steps = [50] # [50, 50, 50, 80, 70]
guidance_scale = [7] # [14, 7, 8, 17, 7.5]
height = 512
width = 512

for num_steps, scale in zip(num_inference_steps, guidance_scale):
    
    print("Generating image with {} inference steps and guidance scale {}".format(num_steps, scale))

    prompt = actual_prompt + ''.join(random.sample(styles, 12))
    print(prompt)
    print("\n\n")

    output = gen_version.predict(
        prompt=prompt,
        num_inference_steps=num_steps,
        guidance_scale=scale,
        height=height,
        width=width,
        num_outputs=4,
    )
    
    # For each output image in the model prediction
    for i, img_link in enumerate(output):
        
        # Display the image
        # image = Image(url=img_link, width=width, unconfined=True)
        # display(image)

        # Download the image 
        response = requests.get(img_link, stream=True)
        filename = str(uuid.uuid4()) + f"num_steps_{num_steps}_scale_{scale}_{i}"
        filepath = OUTPUT_DIR +  filename + ".png"
        with open(filepath, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)

        # Super resolution
        output = upscale_version.predict(
            image=img_link,
            task_type="Real-World Image Super-Resolution-Large",
            noise=15,
        )
        response = requests.get(output, stream=True)
        filepath = OUTPUT_DIR + filename + "_upscaled.png"
        with open(filepath, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)

In [ ]:
# Style transfer into an anime style
model = replicate.models.get("ptran1203/pytorch-animegan")
version = model.versions.get("7d44f1878a07e7b5a32af9727c1f6120cac04203d48f3f7b0432e28fa8e5c6b6")

for image_path in [
    "alex",
    "hugo",
    "brandon",
]:
    output = version.predict(image=f"/home/tren/Pictures/bivy/{image_path}.jpg")
    response = requests.get(output, stream=True)
    filepath = OUTPUT_DIR + filename + "_anime.png"
    with open(filepath, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)